<a href="https://colab.research.google.com/github/yasirsid2004/Bots/blob/main/Binance_Futures_Trading_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import logging

In [3]:
import os

In [ ]:
import logging
import os
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.exceptions import BinanceAPIException, BinanceOrderException

# --- Setup a logger for the Bot ---
# This will log all bot activity, API requests, and errors to a file named 'trading_bot.log'
# and also print informational messages to the console.
log_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('BinanceBot')
logger.setLevel(logging.DEBUG) # Set to DEBUG to capture all levels of logs

# File Handler
file_handler = logging.FileHandler('trading_bot.log')
file_handler.setFormatter(log_formatter)
logger.addHandler(file_handler)

# Console Handler
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO) # Only show INFO and above in the console
console_handler.setFormatter(log_formatter)
logger.addHandler(console_handler)


class TradingBot:
    """
    A class to represent a simple trading bot for Binance Futures Testnet.
    """
    def __init__(self, api_key, api_secret):
        """
        Initializes the TradingBot.
        Args:
            api_key (str): Your Binance API key.
            api_secret (str): Your Binance API secret.
        """
        self.api_key = api_key
        self.api_secret = api_secret

        try:
            # Initialize the client for testnet
            self.client = Client(self.api_key, self.api_secret, testnet=True)
            # IMPORTANT: Set the API URL to the futures testnet URL
            self.client.API_URL = 'https://testnet.binancefuture.com/fapi'
            logger.info("Successfully initialized Binance client for Futures Testnet.")
            self.test_connection()
        except Exception as e:
            logger.error(f"Failed to initialize Binance client: {e}")
            raise

    def test_connection(self):
        """Tests the API connection and credentials."""
        try:
            account_info = self.client.futures_account()
            logger.info("API Connection Successful. Account information received.")
            # Log some account details for verification
            usdt_balance = next((asset['walletBalance'] for asset in account_info['assets'] if asset['asset'] == 'USDT'), None)
            logger.info(f"USDT Wallet Balance: {usdt_balance}")
        except BinanceAPIException as e:
            logger.error(f"API Connection Failed: {e.status_code} - {e.message}")
            raise
        except Exception as e:
            logger.error(f"An unexpected error occurred during connection test: {e}")
            raise

    def place_market_order(self, symbol, side, quantity):
        """
        Places a market order.
        Args:
            symbol (str): The trading pair (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount to trade.
        Returns:
            dict: The order response from the API, or None if failed.
        """
        logger.info(f"Attempting to place MARKET order: {side} {quantity} {symbol}")
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side.upper(),
                type='MARKET',
                quantity=quantity
            )
            logger.info("Successfully placed MARKET order.")
            logger.debug(f"API Response: {order}")
            return order
        except BinanceAPIException as e:
            logger.error(f"API Error placing MARKET order: {e.status_code} - {e.message}")
        except BinanceOrderException as e:
            logger.error(f"Order Error placing MARKET order: {e.status_code} - {e.message}")
        except Exception as e:
            logger.error(f"An unexpected error occurred placing MARKET order: {e}")
        return None

    def place_limit_order(self, symbol, side, quantity, price):
        """
        Places a limit order.
        Args:
            symbol (str): The trading pair (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount to trade.
            price (float): The price at which to place the order.
        Returns:
            dict: The order response from the API, or None if failed.
        """
        logger.info(f"Attempting to place LIMIT order: {side} {quantity} {symbol} @ {price}")
        try:
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side.upper(),
                type='LIMIT',
                timeInForce='GTC',  # Good Till Canceled
                quantity=quantity,
                price=price
            )
            logger.info("Successfully placed LIMIT order.")
            logger.debug(f"API Response: {order}")
            return order
        except BinanceAPIException as e:
            logger.error(f"API Error placing LIMIT order: {e.status_code} - {e.message}")
        except BinanceOrderException as e:
            logger.error(f"Order Error placing LIMIT order: {e.status_code} - {e.message}")
        except Exception as e:
            logger.error(f"An unexpected error occurred placing LIMIT order: {e}")
        return None

    def place_stop_limit_order(self, symbol, side, quantity, price, stop_price):
        """
        Places a stop-limit order (Bonus).
        This is a STOP_MARKET order in the API terminology for futures.
        It triggers a market order when the stopPrice is reached.
        Args:
            symbol (str): The trading pair (e.g., 'BTCUSDT').
            side (str): 'BUY' or 'SELL'.
            quantity (float): The amount to trade.
            price (float): The limit price for the order once triggered.
            stop_price (float): The price at which the order is triggered.
        Returns:
            dict: The order response from the API, or None if failed.
        """
        logger.info(f"Attempting to place STOP-LIMIT order: {side} {quantity} {symbol} @ price {price}, stop {stop_price}")
        try:
            # Note: For futures, a common stop-limit is STOP_MARKET or TAKE_PROFIT_MARKET
            # A true STOP-LIMIT requires both price and stopPrice.
            order = self.client.futures_create_order(
                symbol=symbol,
                side=side.upper(),
                type='STOP', # This becomes a limit order when triggered.
                timeInForce='GTC',
                quantity=quantity,
                price=price, # The limit price
                stopPrice=stop_price # The trigger price
            )
            logger.info("Successfully placed STOP-LIMIT order.")
            logger.debug(f"API Response: {order}")
            return order
        except BinanceAPIException as e:
            logger.error(f"API Error placing STOP-LIMIT order: {e.status_code} - {e.message}")
        except BinanceOrderException as e:
            logger.error(f"Order Error placing STOP-LIMIT order: {e.status_code} - {e.message}")
        except Exception as e:
            logger.error(f"An unexpected error occurred placing STOP-LIMIT order: {e}")
        return None


def get_validated_input(prompt, type_converter, validator=None):
    """Generic function to get and validate user input."""
    while True:
        try:
            value = type_converter(input(prompt))
            if validator is None or validator(value):
                return value
            else:
                logger.warning("Invalid input. Please try again.")
        except ValueError:
            logger.warning("Invalid type. Please enter the correct format.")
        except Exception as e:
            logger.error(f"An error occurred in input: {e}")


def main_cli():
    """Main command-line interface to run the bot."""
    print("\n--- Binance Futures Trading Bot ---")
    print("This bot connects to the Binance Futures TESTNET.")
    print("It does NOT use real money.")

    api_key = os.environ.get('BINANCE_API_KEY')
    api_secret = os.environ.get('BINANCE_API_SECRET')

    if not api_key:
        api_key = input("Enter your Binance Testnet API Key: ").strip()
    if not api_secret:
        api_secret = input("Enter your Binance Testnet API Secret: ").strip()

    try:
        bot = TradingBot(api_key, api_secret)
    except Exception:
        logger.error("Could not start the bot. Please check your API credentials and connection.")
        return

    while True:
        print("\n--- Main Menu ---")
        print("1. Place Market Order")
        print("2. Place Limit Order")
        print("3. Place Stop-Limit Order (Bonus)")
        print("4. Exit")

        choice = get_validated_input("Enter your choice (1-4): ", int, lambda x: 1 <= x <= 4)

        if choice == 4:
            logger.info("Exiting bot.")
            break

        symbol = get_validated_input("Enter Symbol (e.g., BTCUSDT): ", str.upper)
        side = get_validated_input("Enter Side (BUY or SELL): ", str.upper, lambda x: x in ['BUY', 'SELL'])
        quantity = get_validated_input("Enter Quantity (e.g., 0.001): ", float, lambda x: x > 0)

        order_result = None
        if choice == 1: # Market Order
            order_result = bot.place_market_order(symbol, side, quantity)
        elif choice == 2: # Limit Order
            price = get_validated_input("Enter Limit Price: ", float, lambda x: x > 0)
            order_result = bot.place_limit_order(symbol, side, quantity, price)
        elif choice == 3: # Stop-Limit Order
            price = get_validated_input("Enter Limit Price (price when triggered): ", float, lambda x: x > 0)
            stop_price = get_validated_input("Enter Stop Price (trigger price): ", float, lambda x: x > 0)
            order_result = bot.place_stop_limit_order(symbol, side, quantity, price, stop_price)

        if order_result:
            print("\n--- Order Placed Successfully ---")
            for key, value in order_result.items():
                print(f"{key}: {value}")
        else:
            print("\n--- Order Failed to Place ---")
            print("Check 'trading_bot.log' for detailed error information.")


if __name__ == '__main__':
    main_cli()


--- Binance Futures Trading Bot ---
This bot connects to the Binance Futures TESTNET.
It does NOT use real money.
Enter your Binance Testnet API Key: MyTradingBot


In [5]:
%pip install python-binance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 53.3 MB/s eta 0:00:00
